<a href="https://colab.research.google.com/github/jantuitman/deeplearning/blob/main/Betere_vertalingen_maken_van_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install pyyaml
!pip install requests
!pip install deepl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import requests

def fetch_squad_json(url, file_name):
    # Make a GET request to the URL to retrieve the JSON data
    response = requests.get(url)

    # Raise an error if the request was not successful
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data from URL: {response.status_code}")

    # Write the JSON data to a file
    with open(file_name, 'w') as f:
        f.write(response.text)

# Example usage
url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json'
file_name = 'drive/MyDrive/InputData/squad-dev.json'
fetch_squad_json(url, file_name)


In [8]:
filename ='drive/MyDrive/InputData/squad-dev.json'

In [9]:
import json
from IPython.utils.text import indent
with open(filename,'r') as f:
  data = json.load(f)
  item = data['data'][0]
  print(json.dumps(item,indent=4))

{
    "title": "Normans",
    "paragraphs": [
        {
            "qas": [
                {
                    "question": "In what country is Normandy located?",
                    "id": "56ddde6b9a695914005b9628",
                    "answers": [
                        {
                            "text": "France",
                            "answer_start": 159
                        },
                        {
                            "text": "France",
                            "answer_start": 159
                        },
                        {
                            "text": "France",
                            "answer_start": 159
                        },
                        {
                            "text": "France",
                            "answer_start": 159
                        }
                    ],
                    "is_impossible": false
                },
                {
                    "question": "When were the Normans i

In [94]:
import xml.etree.ElementTree as ET
import uuid

def squad_item_to_xml_element(item):
    # Create the root element for the item
    item_el = ET.Element('item')
    item_el.set('id', str(uuid.uuid4()))
    
    title = ET.SubElement(item_el, 'title')
    title.text = item['title']
    paras_el = ET.SubElement(item_el,'paragraphs')
    for para in item['paragraphs']:
        para_el = ET.SubElement(paras_el, 'paragraph')
        para_el.set('id', str(uuid.uuid4()))

        context = ET.SubElement(para_el, 'context')
        context.text = para['context']
        qas = ET.SubElement(para_el,"qas")
        for qa in para['qas']:
            qa_el = ET.SubElement(qas, 'question_answer')
            qa_el.set('id', qa['id'])

            question = ET.SubElement(qa_el, 'question')
            question.text = qa['question']

            answers = ET.SubElement(qa_el, 'answers')
            for answer in qa['answers']:
                ans = ET.SubElement(answers, 'answer')
                ans.text = answer['text']
                ans.set('answer_start', str(answer['answer_start']))
            
            qa_el.set('is_impossible', str(qa["is_impossible"]))

            if 'plausible_answers' in qa:
                plausible_answers = ET.SubElement(qa_el, 'plausible_answers')
                for plausible_answer in qa['plausible_answers']:
                    plausible_ans = ET.SubElement(plausible_answers, 'plausible_answer')
                    plausible_ans.text = plausible_answer['text']
                    plausible_ans.set('answer_start', str(plausible_answer['answer_start']))

    return item_el



In [10]:
import xml.etree.ElementTree as ET

def squad_json_to_xml(json_file, xml_file):
    # Load the JSON data from the file
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Create the XML root element
    root = ET.Element('root')
    data_el = ET.SubElement(root, 'data')

    # Iterate over the data and convert each item to an XML element
    for item in data['data']:
        item_el = squad_item_to_xml_element(item)
        data_el.append(item_el)

    # Write the XML data to the file
    tree = ET.ElementTree(root)
    tree.write(xml_file)


In [14]:
from xml.dom import minidom
def serialize_xml_element(element):
    # Convert the element tree to a string
    xml_str = ET.tostring(element, 'utf-8')

    # Use minidom to format the XML string with proper indentation
    xml_str = minidom.parseString(xml_str).toprettyxml(indent='  ')

    return xml_str

In [95]:
import json
from IPython.utils.text import indent
with open(filename,'r') as f:
  data = json.load(f)
  item = data['data'][0]
  print(serialize_xml_element(squad_item_to_xml_element(item)))

<?xml version="1.0" ?>
<item id="02fbb018-d3f9-4613-a148-f044da805afe">
  <title>Normans</title>
  <paragraphs>
    <paragraph id="d50aa2ec-253b-4c67-b3e6-336f962833a0">
      <context>The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (&quot;Norman&quot; comes from &quot;Norseman&quot;) raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.</context>
      <qas>
        <question_answer id="5

In [83]:
squad_json_to_xml(filename,"drive/MyDrive/InputData/squad-dev.xml")

In [84]:
squad_json_to_xml("drive/MyDrive/InputData/squad-train.json","drive/MyDrive/InputData/squad-train.xml")

In [30]:
import yaml
def get_deepl_api_key():
    with open('drive/MyDrive/InputData/secrets.yml', 'r') as f:
        data = yaml.safe_load(f)
    return data['deepl']['key']    

In [33]:
api_key = get_deepl_api_key()


In [47]:
import deepl
def translate_xml_with_deepl(xml_str, target_language='nl'):
    translator = deepl.Translator(api_key)
    translation = translator.translate_text(xml_str,target_lang=target_language,tag_handling='xml')
    return ET.fromstring(translation.text)

In [48]:

with open(filename,'r') as f:
  data = json.load(f)
  item = data['data'][0]
  text = serialize_xml_element(squad_item_to_xml_element(item))
  translated_text = translate_xml_with_deepl(text)
  print(translated_text)

<Element 'item' at 0x7f5d4562adb0>


In [67]:
def addnext(parent, elem, new_elem):
    index = list(parent).index(elem)
    parent.insert(index + 1, new_elem)


In [101]:
def add_translated_questions_and_answers(original_xml, translated_xml, language = "nl"):
    title = original_xml.find('title')
    translated_title = ET.Element("translated_title")
    translated_title.set('language',language)
    translated_title.text = translated_xml.find('title').text
    addnext(original_xml, title, translated_title)
    
    paragraphs = original_xml.findall('paragraphs/paragraph')
    translated_paragraphs = translated_xml.findall('paragraphs/paragraph')

    for paragraph,translated_paragraph in zip(paragraphs,translated_paragraphs):
      context = paragraph.find('context')
      translated_context = ET.Element("translated_context")
      translated_context.set('language',language)
      translated_context.text = translated_paragraph.find('./context').text
      addnext(paragraph,context,translated_context)
    
    original_qas = original_xml.findall("paragraphs/paragraph/qas/question_answer")
    translated_qas = translated_xml.findall("paragraphs/paragraph/qas/question_answer")
    
    for original_qa, translated_qa in zip(original_qas, translated_qas):
        original_question = original_qa.find("./question")

        translated_question_el = translated_qa.find("./question")
        translated_question = ET.Element('translated_question')
        translated_question.set('language',language)
        translated_question.text = translated_question_el.text
        addnext(original_qa,original_question, translated_question)
        
        
        answers = original_qa.find('answers')
        original_answers = original_qa.findall("./answers/answer")
        translated_answer_els = translated_qa.findall("./answers/answer")
        for original_answer, translated_answer_el in zip(original_answers,translated_answer_els):
            translated_answer = ET.Element('translated_answer')
            translated_answer.set('language',language)
            translated_answer.text = translated_answer_el.text
            addnext(answers, original_answer,translated_answer)

        
        original_plausible_answers = original_qa.findall("./plausible_answers/plausible_answer")
        translated_plausible_answer_els = translated_qa.findall("./plausible_answers/plausible_answer")
        plausible_answers = original_qa.find('plausible_answers')
        for original_plausible_answer, translated_plausible_answer_el in zip(original_plausible_answers, translated_plausible_answer_els):
            translated_plausible_answer = ET.Element('translated_plausible_answer')
            translated_plausible_answer.set('language',language)
            translated_plausible_answer.text = translated_plausible_answer_el.text
            addnext(plausible_answers,original_plausible_answer, translated_plausible_answer) 


    return original_xml

In [96]:
xml = None
translated_xml = None
with open(filename,'r') as f:
  data = json.load(f)
  item = data['data'][0]
  xml = squad_item_to_xml_element(item);
  text = serialize_xml_element(xml)
  translated_xml = translate_xml_with_deepl(text)
  #add_translated_questions_and_answers(xml, translated_xml)
  #print(serialize_xml_element(xml))
  

In [102]:
with open(filename,'r') as f:
  data = json.load(f)
  item = data['data'][0]
  xml = squad_item_to_xml_element(item);
  add_translated_questions_and_answers(xml, translated_xml)
  print(serialize_xml_element(xml))
  

<?xml version="1.0" ?>
<item id="6d635043-6b49-4f54-8655-10dfdda74da4">
  <title>Normans</title>
  <translated_title language="nl">Normandiërs</translated_title>
  <paragraphs>
    <paragraph id="44e8c168-6f39-4256-a312-2b08b0f392b6">
      <context>The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (&quot;Norman&quot; comes from &quot;Norseman&quot;) raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeedin